# 引言
元类(metaclass)能够拦截Python的class语句，让系统每次定义类的时候，都能实现某些特殊的行为。
Python还内置了一种神奇而强大的特性，可以动态地定制属性访问操作。

# #44 用纯属性与修饰器取代旧式的setter与getter方法

从其他编程语言转入Python的开发者，可能想在类里面明确地实现getter与setter方法。

In [1]:
class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms

    def get_ohms(self):
        return self._ohms

    def set_ohms(self, ohms):
        self._ohms = ohms

虽然这些setter与getter用起来很简单，但这并不符合Python的风格。

In [2]:
r0 = OldResistor(50e3)
print('Before:', r0.get_ohms())
r0.set_ohms(10e3)
print('After: ', r0.get_ohms())

Before: 50000.0
After:  10000.0


例如，想让属性值变大或变小，采用这些方法来写会特别麻烦。

In [3]:
r0.set_ohms(r0.get_ohms() - 4e3)
assert r0.get_ohms() == 6e3

在Python中没有必要明确定义setter与getter方法。而是应该从最简单的public属性开始写起，例如像下面这样：

In [4]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0 
    
r1 = Resistor(50e3)
r1.ohms = 10e3

这样就很容易实现原地增减属性值。

In [5]:
r1.ohms += 5e3

将来如果想在设置属性时，实现特别的功能，那么可以先通过`@property`修饰器来封装获取属性的那个方法，并在封装出来的修饰器上面通过setter属性来封装设置属性的那个方法。下面这个新类继承自刚才的`Resistor`类，它允许我们通过设置voltage(电压)来改变current(电流)。为了正确实现这项功能，必须保证设置属性与获取属性所用的那两个方法都跟属性同名。

In [7]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

按照这种写法，给voltage属性赋值会触发同名的setter方法，该方法会根据新的voltage计算本对象的current属性。

In [8]:
r2 = VoltageResistance(1e3)
print(f'Before: {r2.current:.2f} amps')
r2.voltage = 10
print(f'After:  {r2.current:.2f} amps')

Before: 0.00 amps
After:  0.01 amps


为属性指定setter方法还可以用来检查调用方所传入的值在类型与范围上是否符合要求。例如，下面这个`Resistor`子类可以确保用户设置的电阻值总是大于0的。

In [10]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms

给这个类设置无效电阻值，程序会抛出异常。

In [11]:
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: ohms must be > 0; got 0

如果构造时所用的值无效，那么同样会触发异常。

In [12]:
BoundedResistance(-5)

ValueError: ohms must be > 0; got -5

之所以会出现这种效果，是因为子类的构造器(`BoundedResistance.__init__`)会调用超类的构造器(`Resistor.__init__`)，而超类的构造器会把`self.ohms`设置成`-5`。
于是就会触发`BoundedResistance`里面的`@ohms.setter`方法，该方法立刻发现属性值无效，所以程序在对象还没有构造完之前，就会抛出异常。

我们还可以利用`@property`阻止用户修改超类中的属性。

In [13]:
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError('Ohms is immutable')
        self._ohms = ohms

构造好对象之后，如果试图给属性赋值，那么程序就会抛出异常。

In [14]:
r4 = FixedResistance(1e3)
r4.ohms = 2e3

AttributeError: Ohms is immutable

用`@property`实现setter和getter时，还应该注意不要让对象产生反常行为。例如，不要在某属性的getter方法里面设置其他属性的值。

In [15]:
class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

假如在获取属性的getter方法里修改了其他属性的值，那么用户查询这个属性时，就会觉得相当奇怪。无法理解为什么另外一个属性会在他查询这个属性时发生变换。

In [16]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print(f'Before: {r7.voltage:.2f}')
r7.ohms
print(f'After:  {r7.voltage:.2f}')

Before: 0.00
After:  0.10


最好的办法是，只在`@property.setter`方法里面修改状态，而且只应该修改对象之中与当前属性有关的状态。同时还要注意不要产生让调用者感到意外的一些副作用，例如，不要动态地引入模块，不要运行速度较慢的辅助函数，不要做I/O等等。
类的属性用起来应该跟其他Python对象一样方便切快捷。如果确实要执行比较复杂或比较缓慢的操作，那么应该用普通的方法来做，而不是应该把这些操作放在获取及设置属性的这两个方法里面。

`@property`最大的缺点是，通过它而编写的获取即属性设置方法只能由子类共享。与此无关的类不能共用这份逻辑。但是没关系，Python还支持描述符，我们可以利用这种机制把早前编写的属性获取与属性设置逻辑复用到其他许多地方。

# #45 考虑用@property实现新的属性访问逻辑，不要急着重构原有的代码

Python内置的`@property`修饰器使开发者很容易就能实现出灵活的逻辑，它还有一种更为高级的用法，也很常见。就是把简单的数值属性迁移成那种实时计算的属性。这个用法可以确保，按照旧写法来访问属性的代码依然有效。
`@property`可以说是一种重要的缓冲机制，使开发者能够逐渐改善接口而不影响已经写好的代码。

例如，下面我们用普通的Python对象实现带有配额(quota)的漏桶(leaky bucket)。这个类可以记录当前的配额以及这份配额在多才时间内有效。

In [6]:
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0
    
    def __repr__(self):
        return f'Bucket(quota={self.quota})'

漏桶算法要求在添加配额时，不能把已有的额度带到下一个时段。

In [3]:
def fill(bucket, amount):
    now = datetime.now()
    if (now  - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

如果想使用额度，那么首先必须确保漏桶当前所剩的配额是足够用的。

In [4]:
def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

现在我们来使用这个类。首先填额度：

In [7]:
bucket = Bucket(60)
fill(bucket, 100)
bucket

Bucket(quota=100)

然后根据自己的需要使用额度：

In [8]:
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')

bucket

Had 99 quota


Bucket(quota=1)

这样用下去，最终会遇到额度不够的情况。从这时开始，额度就不会再变了。

In [9]:
if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
bucket

Not enough for 3 quota


Bucket(quota=1)

这种实现方式有个问题，就是没办法知道第一次填充漏桶时，给它分配的额度。我们只知道额度会越用越少直到不够位置。
如果当前这段时间内的额度已经降到0，那么不管你想使用多少额度，`deduct`函数都会返回`False`，除非通过`fill`函数再往里面补充额度。
所以，当`dedcut`函数返回`False`时，了解这究竟是因为`Bucket`没有足够的额度可以扣减，还是说它一开始根本就没有分配到任何额度，很重要。

为了解决这个问题，可以修改这个类，把当前时间段内的初始额度与已经使用的额度明确记录下来。

In [12]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
    
    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}) ', f'quota_consumed={self.quota_consumed}')
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

同时，为了让用户能像使用原来的`Bucket`类那样使用这个新类。我们下面用这个`@property`方法根据刚才设计的那两个属性实时计算漏桶目前的水位。

In [13]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
    
    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}) ', f'quota_consumed={self.quota_consumed}')
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

然后，我们实现下面这个方法，用来处理quota属性的赋值操作。采用旧式的`fill`与`deduct`函数来增减额度的那些代码依然可以正常运作，因为那两个函数在修改额度时会触发这个新方法，笔者在的代码里对相关情况做了特殊处理。

In [14]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
    
    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}) ', f'quota_consumed={self.quota_consumed}')
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0 :
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            assert amount > 0
            self.quota_consumed = delta

按照旧的用法来使用新的漏桶，依然可以得到正确的结果。

In [17]:
bert = NewBucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
    
print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')

print('Still', bucket)

Initial Bucket(quota=1)
Filled Bucket(quota=101)
Had 99 quota
Now Bucket(quota=2)
Not enough for 3 quota
Still Bucket(quota=2)


这个方案的最大好处是，原来根据`Bucket.quota`所写的那些代码可以继续沿用，而且无须考虑`Bucket`现在已经换成了新的`NewBucket`。

# #46 用描述符来改写需要复用的@property方法

Python内置的`@property`机制的最大确点就是不方便复用。例如，我们要编写一个类来记录学生的家庭作业成绩，而且要确保设置的成绩位于0到100之间。

In [2]:
class Homework:
    def __init__(self):
        self._grade = 0
    
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError(f'Grade must be betwwen 0 and 100')
        self._grade = value

受`@property`修饰的属性用起来很简单。

In [3]:
galileo = Homework()
galileo.grade = 95

假设，我们还需要写一个类记录学生的考试成绩，而且要把每科的成绩分别记录下来。

In [4]:
class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
    
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError(f'Grade must be betwwen 0 and 100')


这样很麻烦，因为每科的成绩都需要一套`@property`方法，而且其中设置属性值的那个方法还必须调用`_check_grade`验证新值。

In [5]:
class Exam:
    def __init__(self):
        self._writing_grade = 0
        self._math_grade = 0
    
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError(f'Grade must be betwwen 0 and 100')
    
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value
    
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value


这样写不仅麻烦，而且无法复用。
在Python里，这样的功能最好通过描述符(descriptor)实现。描述符协议规定了程序应该如何处理属性访问操作。充当描述符的那个类能够实现`__get__`与`__set__`方法，这样其他类就可以共用这个描述符所实现的逻辑而无须把这套逻辑分别重写一遍。

下面重新定义`Exam`类，这次我们采用类级别的属性来实现每科成绩的访问功能，这些属性指向下面这个`Grade`类的实例，而这个`Grade`类则实现刚才提到的描述符协议。

In [6]:
class Grade:
    def __get__(self, instance, instance_type):
        pass
    def __set__(self, instance, value):
        pass

class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    

在解释`Grade`类的工作原理之前，我们首先要知道，当程序访问`Exam`实例的某个属性时，Python如何将访问操作派发到`Exam`类的描述符属性上面。例如，如果要给`Exam`实例的`writing_grade`属性赋值：

In [7]:
exam = Exam()
exam.writing_grade = 40

那么，Python会把这次赋值操作转译为：

In [8]:
Exam.__dict__['writing_grade'].__set__(exam, 40)

获取这个属性时也一样：

In [9]:
exam.writing_grade

Python会转译为：

In [10]:
Exam.__dict__['writing_grade'].__get__(exam, Exam)

这样的转译效果是由`object`的`__getattribute__`方法促成的。简单地说，就是当`Eaxm`实例里面没有名为`writing_grade`的属性时，Python会转而在类的层面查找，查询`Eaxm`类里面有没有这样一个属性。如果有，而且还实现了`__get__`与`__set__`方法后，那么系统就认定你想通过描述符协议定义的这个属性的访问行为。

知道了这条规则之后，我们来尝试把`Homework`类早前用`@property`实现的成绩验证逻辑搬到`Grade`描述符里面。

In [11]:
class Grade:
    def __init__(self):
        self._value = 0
    
    def __get__(self, instance, instance_type):
        return self._value
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(f'Grade must be betwwen 0 and 100')
        self._value = value


这样写其实不对，而且会让程序出现混乱。但在同一个`Exam`实例上面访问不同的属性是没有问题的。

In [12]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


但是，在不同的`Exam`实例上分别访问同一个属性却会看到奇怪的结果。

In [13]:
second_exam = Exam()
second_exam.writing_grade = 75

print(f'Second {second_exam.writing_grade} is right')
print(f'First {first_exam.writing_grade} is wrong;'
     f'should be 82')


Second 75 is right
First 75 is wrong;should be 82


出现这个问题的原因在于，这些`Eaxm`实例之中的`writing_grade`属性实际上是在共享同一个`Grade`实例。
为了解决此问题，我们必须把每个`Exam`实例在这个属性上面的取值都记录下来。可以通过字典实现每个实例的状态保存。

In [14]:
class Grade:
    def __init__(self):
        self._values = {}
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(f'Grade must be betwwen 0 and 100')
        self._values[instance] = value


这种实现方案很简单，而且能得到正确结果，但仍然有一个缺陷，就是会泄露内存。

为了解决这个问题，我们可以求助于Python内置的`weakref`模块。该模块里有一种特殊的字典，名为`WeakKeyDictionary`，它可以取代刚才实现`_values`时所用的普通字典。
这个字典的特殊之处在于：如果运行时系统发现，指向`Eaxm`实例的引用只剩一个，而这个引用又是由`WeakKeyDictionary`的键所发起的，那么系统会将该引用从这个特殊的字典里删掉，于是指向那个`Exam`实例的引用数量就会降为0。

In [15]:
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()
    
    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(f'Grade must be betwwen 0 and 100')
        self._values[instance] = value


用这种字典改写`Grade`描述符之后，`Exam`就能正常运作了。

In [16]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75
print(f'First {first_exam.writing_grade} is right')
print(f'Second {second_exam.writing_grade} is right')


First 82 is right
Second 75 is right


# 针对惰性属性使用__getattr__、__getattribute__及__setattr__

假设我们想把数据库中的记录表示为Python对象，数据库有它自己的模式(schema)，而程序在把记录表示成对象时，必须知道数据库是按照什么样的模式来组织这些记录的。

这种动态的行为可以通过名为`__getattr__`的特殊方法来实现。如果类中定义了此方法，那么每当访问该类对象的属性，而且实例字典里又找不到这个属性时，系统就会触发`__getattr__`方法。

In [1]:
class LazyRecord:
    def __init__(self):
        self.exists = 5
    def __getattr__(self, name):
        value = f'Value for {name}'
        setattr(self, name, value)
        return value

In [2]:
data = LazyRecord()
print('Before: ', data.__dict__) # 此时并没有foo这个属性
print('foo:  ',data.foo)
print('After: ', data.__dict__) # 此时多了foo这个属性

Before:  {'exists': 5}
foo:   Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


下面我们通过子类给LazyRecord增加日志功能，用来观察程序在什么样的情况下才会调用`__getattr__`方法。

In [3]:
class LoggingLazyRecord(LazyRecord):
    def __getattr__(self, name):
        print(f'* Called __getattr__({name!r}), populating instance dictionary')
        result = super().__getattr__(name)
        print(f'* Returning {result!r}')
        return result

d